In [64]:
import os
import time
import datetime
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from src.main import prepare_data as prep_demo_data
from src.config import Config
# from src.transformer import Transformer
from src.hacked_transformer import Transformer

from src.utils import CustomSchedule
from src.utils import LossObject
from src.utils import create_masks
from src.reids_data_loader.data_generator import DataGenerator

In [3]:
def prepare_i2a_data():
    
    pass
    
def prepare_data(config, dataset_type="I2A"):
    """
    PARAMS:
    dataset_type (str): what dataset to prepare on of ["I2A", "PT2EN"]
    """
    if dataset_type == "I2A":
        train_dataset, val_dataset, tokenizer_in, tokenizer_out = prepare_i2a_data(config)
        return train_dataset, val_dataset, tokenizer_in, tokenizer_out

    if dataset_type == "PT2EN":
        (train_dataset, val_dataset, 
         tokenizer_in, tokenizer_out) = prep_demo_data(config, 
                                                       "../data/en_tokenizer", 
                                                       "../data/pt_tokenizer")
        return train_dataset, val_dataset, tokenizer_in, tokenizer_out
    



def translate(sentence, plot=''):
    result, attention_weights = evaluate(sentence)

    predicted_sentence = tokenizer_en.decode([i for i in result 
                                            if i < tokenizer_en.vocab_size])  

    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(predicted_sentence))
  
    if plot:
        plot_attention_weights(attention_weights, sentence, result, plot)
        



In [4]:
config = Config()

In [5]:
# demo_train_dataset, demo_val_dataset, demo_tokenizer_en, demo_tokenizer_pt = prepare_data(config, dataset_type="PT2EN")
# print("Complete")

# Load MNIST keras-style

In [36]:
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils import to_categorical


In [91]:
IMG_SHAPE = (28, 28, 1)
NUM_CLASSES = 10
img_rows = IMG_SHAPE[0]
img_cols = IMG_SHAPE[1]

## Load Data:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# data = np.load(path_data)
# (x_train, y_train), (x_test, y_test) = (data["x_train"], data["y_train"]), (data["x_test"], data["y_test"])

# Normalize and Reshape Data:
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.

# y_train = to_categorical(y_train, NUM_CLASSES)
# y_test = to_categorical(y_test, NUM_CLASSES)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# input_shape = (img_rows, img_cols, 1)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)
print()

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)
(10000,)



In [94]:
def mnist_numpy_generator(epochs=3, batch_size=30):
#     temp = np.arange(90).reshape(10, 3, 3, 1)
#     print(len(temp))
#     print(temp.shape)
# #     print(temp)

#     temp = temp.reshape(5, 2, -1)
    
#     for ii in temp:
#         print(type(ii))
#         print(ii)
#         print("---")

    ## Flatten and batch images
    ## Batchsize must be a factor of len(x_train)
#     xx = x_train.reshape(len(x_train)//batch_size, batch_size, 1, -1)
#     yy = y_train.reshape(len(y_train)//batch_size, batch_size, 1, -1)
    xx = x_train.reshape(len(x_train)//batch_size, batch_size, -1)
    yy = y_train.reshape(len(y_train)//batch_size, batch_size, -1)
    for _ in range(epochs):
        for batch in zip(xx, yy):
            yield batch
        


In [178]:
def fake_data_gen():
    L = 20 # len dataset
    x_size = (3, 3)
    y_size = 4
    xx_data_range = 300
    yy_data_range = 3
    batch_size = 2 # must divide evenly into L
    batch_num = L//batch_size
    xx = np.random.randint(0, xx_data_range, (L, x_size[0], x_size[1]))
    xx[:, :, 0] = 0
    yy = np.random.randint(0, yy_data_range, (L, y_size))
#     yy = to_categorical(yy, yy_data_range)
    
    def _add_start_stop_tokens(data, data_range, batch_num, batch_size):
        data = data.reshape(batch_num, batch_size, -1)
        temp_start = data_range * np.ones((batch_num, batch_size, 1))
        temp_stop = data_range * np.ones((batch_num, batch_size, 1)) + 1

        return np.concatenate([temp_start, data, temp_stop], axis=2)

    for item in zip(_add_start_stop_tokens(xx, xx_data_range, batch_num, batch_size), 
                    _add_start_stop_tokens(yy, yy_data_range, batch_num, batch_size)):
        yield item

next(fake_data_gen())

(array([[300.,   0.,  47., 174.,   0., 284., 168.,   0.,   8., 296., 301.],
        [300.,   0., 117.,   9.,   0., 266.,  51.,   0., 256., 295., 301.]]),
 array([[3., 0., 1., 0., 2., 4.],
        [3., 1., 0., 0., 1., 4.]]))

In [95]:
for ii, item in enumerate(mnist_numpy_generator()):
    if ii > 5: break
    print(type(item), item[0].shape, item[1].shape)

<class 'tuple'> (30, 784) (30, 1)
<class 'tuple'> (30, 784) (30, 1)
<class 'tuple'> (30, 784) (30, 1)
<class 'tuple'> (30, 784) (30, 1)
<class 'tuple'> (30, 784) (30, 1)
<class 'tuple'> (30, 784) (30, 1)


In [96]:
sample_item[1][0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [97]:
def mnist_train_transformer(model, config, data=None):
    """
    Transformer training code
    """
    log_dir = config.log_dir

#     BUFFER_SIZE = config.BUFFER_SIZE
#     BATCH_SIZE = config.BATCH_SIZE
    EPOCHS = 1 # config.EPOCHS
#     MAX_LENGTH = config.MAX_LENGTH

    # Hyperparameters
    num_layers = config.num_layers
    d_model = config.d_model
    dff = config.dff
    num_heads = config.num_heads
    dropout_rate = config.dropout_rate

    print("preparing data ...")
    train_dataset = data

    transformer = model

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy')

    learning_rate = CustomSchedule(d_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate,
                                         beta_1=0.9,
                                         beta_2=0.98,
                                         epsilon=1e-9)
    loss_function = LossObject().loss_function

    checkpoint_path = "./checkpoints/train"

    ckpt = tf.train.Checkpoint(transformer=transformer,
                               optimizer=optimizer)

    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

    # if a checkpoint exists, restore the latest checkpoint.
    if ckpt_manager.latest_checkpoint:
        # ckpt.restore(ckpt_manager.latest_checkpoint)
        ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
        print ('Latest checkpoint restored!!')

    ## set up logging of Tensorboard events
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = os.path.join(log_dir, "events", current_time, "train")
    # test_log_dir = log_dir + current_time + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    # test_summary_writer = tf.summary.create_file_writer(test_log_dir)

    train_step_signature = [
        tf.TensorSpec(shape=(None, None), dtype=tf.int64),
        tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    ]

    @tf.function(input_signature=train_step_signature)
    def train_step(inp, tar):
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        (enc_padding_mask,
         combined_mask,
         dec_padding_mask) = create_masks(inp, tar_inp)

        with tf.GradientTape() as tape:
            predictions, _ = transformer(inp, tar_inp,
                                         True,
                                         enc_padding_mask,
                                         combined_mask,
                                         dec_padding_mask)
            loss = loss_function(tar_real, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients,
                                      transformer.trainable_variables))

        train_loss(loss)
        train_accuracy(tar_real, predictions)


    # def test_step(model, x_test, y_test):
    #     predictions = model(x_test)
    #     loss = loss_object(y_test, predictions)
    #
    #     test_loss(loss)
    #     test_accuracy(y_test, predictions)


    for epoch in range(EPOCHS):
        start = time.time()

        train_loss.reset_states()
        train_accuracy.reset_states()

        # inp -> portuguese, tar -> english
        for (batch, (inp, tar)) in enumerate(train_dataset):
            train_step(inp, tar)

            ## Tensorboard event logging
            ### QUESTION: should this be less often?
            with train_summary_writer.as_default():
                tf.summary.scalar('loss',
                                  train_loss.result(),
                                  step=epoch)
                tf.summary.scalar('accuracy',
                                  train_accuracy.result(),
                                  step=epoch)

            if batch % 50 == 0:
                print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                       epoch + 1, batch, train_loss.result(), train_accuracy.result()))

        if (epoch + 1) % 5 == 0:
            ckpt_save_path = ckpt_manager.save()
            print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

        print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                    train_loss.result(),
                                                    train_accuracy.result()))

        print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

    print("We got to here!!!")

In [98]:
def _evaluate(inp_sentence, model):
    transformer = model
    
#     start_token = [mnist_input_vocab_size]
#     end_token = [mnist_input_vocab_size + 1]

#     # inp sentence is portuguese, hence adding the start and end token
#     inp_sentence = start_token + inp_sentence + end_token

    encoder_input = tf.expand_dims(inp_sentence, 0)
    print(encoder_input.shape)
#     encoder_input = inp_sentence

    # as the target is english, the first word to the transformer should be the
    # english start token.
    decoder_input = [mnist_target_vocab_size]
    output = tf.expand_dims(decoder_input, 0)

#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#                                                             encoder_input, output)
    
    for i in range(MAX_LENGTH):
#     for i in range(3):

        if i % 20 == 0:
            print("at step {} of {}".format(i, MAX_LENGTH))
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input, 
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if predicted_id == mnist_target_vocab_size: # tokenizer_en.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights
    
        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


# Break down training function
## Call transformer

In [177]:
from src.hacked_transformer import Transformer
from src.hacked_transformer import Encoder
from src.hacked_transformer import Decoder


MAX_LENGTH = config.MAX_LENGTH

num_layers = config.num_layers
d_model = 9 # config.d_model
num_heads = 3 #config.num_heads
dff = config.dff
dropout_rate = config.dropout_rate

mnist_input_vocab_size = 258
mnist_target_vocab_size = 10

# data = mnist_numpy_generator()
data = fake_data_gen()

print("Creating Transformer")
transformer = Transformer(num_layers, 
                          d_model, 
                          num_heads, 
                          dff,
                          mnist_input_vocab_size, 
                          mnist_target_vocab_size,
                          pe_input=800, # mnist_input_vocab_size,
                          pe_target=5, # mnist_target_vocab_size,
                          rate=dropout_rate)

sample_item = next(data)

Creating Transformer
(3, 2, 9)
(3, 2, 9)


In [175]:
# mnist_train_transformer(transformer, config, data)

In [176]:
temp_inp = sample_item[0][0]
temp_out = sample_item[0][1]

temp_inp = tf.expand_dims(temp_inp, 0)

# temp_decoder_input = [11]
temp_out = tf.expand_dims(temp_out, 0)

#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#                                                             encoder_input, output)
    
aa, bb, cc = create_masks(temp_inp, temp_out)

temp_enc = Encoder(num_layers=3, 
                   d_model=9, 
                   num_heads=3, 
                   dff=16, 
                   input_vocab_size=258,
#                    target_vocab_size=11, 
                   maximum_position_encoding=800, 
#                    pe_target=3, 
                   rate=0.1)
temp_enc(temp_input, False, bb)

InvalidArgumentError: slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: encoder_33/strided_slice/

In [109]:
temp_targ_voc_size = 10
temp_d_model = 4

temp_input = np.random.randint(0, 10, (1, 1, 8))
print(temp_input.shape)
print(temp_input)
# print(temp_input[0, 0, 4])

temp_out = tf.keras.layers.Embedding(temp_targ_voc_size, temp_d_model)(temp_input).numpy()
print(temp_out.shape)
temp_out

(1, 1, 8)
[[[9 9 4 4 2 7 9 6]]]
(1, 1, 8, 4)


array([[[[-0.01878595,  0.04641351,  0.03351081,  0.00574498],
         [-0.01878595,  0.04641351,  0.03351081,  0.00574498],
         [ 0.02236458, -0.03413758, -0.01948105, -0.04260851],
         [ 0.02236458, -0.03413758, -0.01948105, -0.04260851],
         [ 0.04038406, -0.02031455,  0.00128482, -0.03006921],
         [-0.00227303, -0.00391444, -0.03188553, -0.03826999],
         [-0.01878595,  0.04641351,  0.03351081,  0.00574498],
         [-0.03483127,  0.00991713, -0.0410812 ,  0.01923412]]]],
      dtype=float32)

In [110]:




# # inp, target = next(data)
inp, tar_inp = sample_item


# (enc_padding_mask,
#  combined_mask,
#  dec_padding_mask) = create_masks(inp, tar_inp)

print(inp.shape, tar_inp.shape)


inp, tar_inp = inp[0], tar_inp[0]
print(inp.shape, tar_inp.shape)

# (enc_padding_mask,
#  combined_mask,
#  dec_padding_mask) = create_masks(inp, tar_inp)

# print(enc_padding_mask.shape,
#  combined_mask.shape,
#  dec_padding_mask.shape)


temp = _evaluate(inp, transformer)

print("Done")

(30, 784) (30, 1)
(784,) (1,)
(1, 784)
at step 0 of 400


InvalidArgumentError: Incompatible shapes: [1,6,9] vs. [1,5,9] [Op:AddV2] name: transformer_19/decoder_18/add/

In [ ]:
# inp, target = next(data)
inp, tar_inp = next(data)

(enc_padding_mask,
 combined_mask,
 dec_padding_mask) = create_masks(inp, tar_inp)

print(inp.shape, tar_inp.shape)

print(enc_padding_mask.shape,
 combined_mask.shape,
 dec_padding_mask.shape)


predictions, _ = transformer(inp, tar_inp,
                             True,
                             enc_padding_mask,
                             combined_mask,
                             dec_padding_mask)